In [ ]:
from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader
from dep_tools.grids import PACIFIC_GRID_10

from dep_tools.s2_utils import mask_clouds

from odc.stac import configure_rio
import folium

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Set up rasterio
configure_rio(cloud_defaults=True, aws={"aws_unsigned": True})

In [ ]:
# Study site configuration
# item_id = "63,20"  # West viti levu
# datetime="2024-08-03"

# item_id = "32,30"  # Cloudy solomons area
# datetime = "2024-09-21"

# item_id = "22,34"  # PNG highlands
# datetime = "2024-09-23"

item_id = "52,24"  # North of Vanuatu
datetime = "2024-09"

# item_id = "56,49"  # Atoll with cloud mask issues
# datetime = "2024-08-05"

# And get the study site
tile_index = tuple(int(i) for i in item_id.split(","))
geobox = PACIFIC_GRID_10.tile_geobox(tile_index)

# Load low-res if you want it faster
# geobox = geobox.crop(100)

geobox.explore()

In [ ]:
# Search for data
searcher = PystacSearcher(
    catalog="https://earth-search.aws.element84.com/v1",
    collections=["sentinel-2-c1-l2a"],
    datetime=datetime,   
)

items = searcher.search(area=geobox)
print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    bands=["scl", "red", "blue", "green", "cloud"],
    chunks=dict(time=1, x=3201, y=3201),
    groupby="solar_day",
    fail_on_error=False,
)

# Run the load process, which is lazy-loaded
data = loader.load(items, areas=geobox)
data

In [ ]:
one = data.isel(time=-1).squeeze().compute()
one[["red", "green", "blue"]].to_array().plot.imshow(size=10, vmin=1000, vmax=3000)

In [ ]:
from xarray import DataArray
from typing import Iterable, Tuple
from odc.algo import mask_cleanup, erase_bad

def mask_clouds(
    xr: DataArray,
    filters: Iterable[Tuple[str, int]] | None = None,
    keep_ints: bool = False,
    return_mask: bool = False,
    include_cirrus: bool = True,
) -> DataArray:
    # NO_DATA = 0
    SATURATED_OR_DEFECTIVE = 1
    # DARK_AREA_PIXELS = 2
    CLOUD_SHADOWS = 3
    # VEGETATION = 4
    # NOT_VEGETATED = 5
    # WATER = 6
    # UNCLASSIFIED = 7
    CLOUD_MEDIUM_PROBABILITY = 8
    CLOUD_HIGH_PROBABILITY = 9
    THIN_CIRRUS = 10
    # SNOW = 11

    mask_list = [
        SATURATED_OR_DEFECTIVE,
        CLOUD_SHADOWS,
        CLOUD_MEDIUM_PROBABILITY,
        CLOUD_HIGH_PROBABILITY,
    ]

    if include_cirrus:
        mask_list.append(THIN_CIRRUS)

    cloud_mask = xr.scl.isin(mask_list)

    if filters is not None:
        cloud_mask = mask_cleanup(cloud_mask, filters)

    if keep_ints:
        masked = erase_bad(xr, cloud_mask)
    else:
        masked = xr.where(~cloud_mask)

    if return_mask:
        return masked, cloud_mask
    else:
        return masked


_, mask = mask_clouds(one, filters=[("dilation", 3), ("erosion", 2)], return_mask=True)
_, mask_new = mask_clouds(one, filters=[("erosion", 3), ("dilation", 6)], return_mask=True)
_, mask_no_mask_cirrus = mask_clouds(one, filters=[("erosion", 3), ("dilation", 6)], return_mask=True, include_cirrus=False)
_, no_filter = mask_clouds(one, return_mask=True)

In [ ]:
m = folium.Map(location=geobox.geographic_extent.centroid.coords[0][::-1], zoom_start=10)

one.odc.to_rgba(vmin=1000, vmax=3000).odc.add_to(m, name="RGB")
mask.where(mask != 0).odc.add_to(m, name="Cloud Mask (old)", vmin=0, vmax=1)
mask_new.where(mask_new != 0).odc.add_to(m, name="Cloud Mask (new)", vmin=0, vmax=1)
mask_no_mask_cirrus.where(mask_no_mask_cirrus != 0).odc.add_to(m, name="Cloud Mask (keep cirrus)", vmin=0, vmax=1)
no_filter.where(no_filter != 0).odc.add_to(m, name="Cloud Mask (no filter)", vmin=0, vmax=1)
one.cloud.odc.add_to(m, name="Cloud Probability")
one.scl.odc.add_to(m, name="SCL")

folium.LayerControl().add_to(m)

m

In [ ]:
m.save("cloudtesting_fiji.html")

In [ ]:
one.scl.odc.write_cog("scl.tif")

In [ ]:
!pwd